In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models import ResNet18_Weights
from sklearn.model_selection import train_test_split
from torchvision import models
import matplotlib.pyplot as plt
import random

## 1. Hàm tiền xử lí dữ liệu ảnh

In [ ]:
def preprocess_image(
    path, target_size=96, median_kernel_size=5, adaptive_block_size=17, adaptive_C=5
):
    if adaptive_block_size % 2 == 0 or adaptive_block_size < 3:
        adaptive_block_size = max(3, adaptive_block_size + 1)
        print(f"Adjusted adaptive_block_size to {adaptive_block_size} (must be odd)")
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None

    img = cv2.medianBlur(cv2.resize(img, (256, 256)), median_kernel_size)
    binary = cv2.adaptiveThreshold(
        img,
        255,  # Use gray_filtered here
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        adaptive_block_size,
        adaptive_C,
    )
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 50]
    if not contours:
        return None
    cnt = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)
    digit = binary[y : y + h, x : x + w]

    size_max = max(w, h)
    square = np.zeros((size_max, size_max), dtype=np.uint8)
    x_offset = (size_max - w) // 2
    y_offset = (size_max - h) // 2
    square[y_offset : y_offset + h, x_offset : x_offset + w] = digit

    resized = cv2.resize(
        square, (target_size, target_size), interpolation=cv2.INTER_AREA
    )
    normalized = resized.astype("float32") / 255.0
    normalized = normalized.reshape(target_size, target_size, 1)
    return np.expand_dims(normalized, axis=0)

In [ ]:
# img = preprocess_image("../dataset/raw_dataset/0/005949_0_3982.JPG")
# img_np = img.squeeze()
# plt.figure(figsize=(3, 3))
# plt.subplot(1, 1, 1)
# plt.imshow(img_np, cmap='gray')
# plt.tight_layout()
# plt.show()

## 2. Class custom dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, target_size=96):
        self.valid_data = []
        for path, label in zip(image_paths, labels):
            img = preprocess_image(path, target_size)
            if img is not None:
                self.valid_data.append((img, label))

    def __len__(self):
        return len(self.valid_data)

    def __getitem__(self, idx):
        img, label = self.valid_data[idx]
        img_tensor = (
            torch.tensor(img, dtype=torch.float32).permute(0, 3, 1, 2).squeeze(0)
        )
        return img_tensor, label

## 3. Hàm load dữ liệu

In [ ]:
def load_image_paths_and_labels(root_dir):
    image_paths, labels = [], []
    for label in os.listdir(root_dir):
        label_path = os.path.join(root_dir, label)
        if not os.path.isdir(label_path):
            continue
        for filename in os.listdir(label_path):
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                image_paths.append(os.path.join(label_path, filename))
                labels.append(int(label))
    return image_paths, labels

## 4. Hàm hiển thị ảnh mẫu

In [ ]:
def show_first_n_images(dataset, n=5):
    plt.figure(figsize=(10, n))
    for i in range(n):
        img_tensor, label = dataset[i]
        img_np = img_tensor.squeeze().numpy()
        plt.subplot(n, 10, i + 1)
        plt.imshow(img_np, cmap="gray")
        plt.title(f"Label: {label}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

## 5. Hàm train model

In [ ]:
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train loss: {total_loss / len(loader):.4f}")

## 6. Hàm test độ chính xác (acc) của model

In [ ]:
def test(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = 100 * correct / total
    print(f"Test loss: {total_loss / len(loader):.4f} - Accuracy: {acc:.2f}%")

## 7. Hàm dự đoán ảnh chữ viết tay từ thư mục tập test

In [ ]:
def predict_digits_from_folder(
    model, input_folder, output_csv, device="cpu", target_size=96
):
    model.eval()
    results = []
    i = 1
    for filename in os.listdir(input_folder):
        print(i)
        i += 1
        if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
            print(f"⚠️ Ảnh không được hỗ trợ: {filename}. Random kết quả.")
            results.append([filename, random.randint(0, 9)])
            continue
        path = os.path.join(input_folder, filename)
        img = preprocess_image(path, target_size)
        if img is None:
            print(f"⚠️ Ảnh lỗi: {filename}. Random kết quả.")
            results.append([filename, random.randint(0, 9)])
            continue
        img_tensor = (
            torch.tensor(img, dtype=torch.float32).permute(0, 3, 1, 2).to(device)
        )
        with torch.no_grad():
            output = model(img_tensor)
            pred = output.argmax(dim=1).item()
        results.append([filename, pred])
    pd.DataFrame(results, columns=["filename", "predicted_label"]).to_csv(
        output_csv, index=False
    )
    print(f"✅ Ghi kết quả vào: {output_csv}")

## 8. Configuration

In [ ]:
TRAIN_SET_DIR = "../dataset/train_set"
TEST_SET_DIR = "../dataset/test_set"
OUTPUT_CSV = "../pred.csv"
EPOCH = 20
TARGET_SIZE = 192

## 9. Chạy chương trình

### 9.1. Load dữ liệu

In [ ]:
# Load và chia dữ liệuuuuu
image_paths, labels = load_image_paths_and_labels(TRAIN_SET_DIR)
# x_train, x_test, y_train, y_test = train_test_split(image_paths, labels, test_size=0.1, stratify=labels, random_state=42)

# Tạo Dataset
train_dataset = CustomDataset(image_paths, labels, target_size=TARGET_SIZE)
# test_dataset = CustomDataset(x_test, y_test, target_size=96)

In [ ]:
# Hiển thị n ânh mẫu đầu tiên
show_first_n_images(train_dataset, n=100)

### 9.2. Train

In [ ]:
# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64)

# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)
model = resnet18.to(device)

# Optimizer & Loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train và Test
for epoch in range(EPOCH):
    print(f"\nEpoch {epoch + 1}")
    train(model, train_loader, criterion, optimizer, device)
    # test(model, test_loader, criterion, device)

### 9.3. Dự đoán

In [ ]:
predict_digits_from_folder(
    model, TEST_SET_DIR, OUTPUT_CSV, device=device, target_size=TARGET_SIZE
)